# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 70 new papers today
          28 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/27 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2211.15665


/tmp/ipykernel_2231/4030337529.py:34: LatexWarning: 2211.15665 did not run properly
not a gzip file
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


Retrieving document from  https://arxiv.org/e-print/2211.15676


extracting tarball to tmp_2211.15676...

...

 done.
Retrieving document from  https://arxiv.org/e-print/2211.16388


/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Found documentclass in tmp_2211.16382/paper.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'divergence' from 'tmp_2211.16382/divergence.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'lmc' from 'tmp_2211.16382/lmc.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex

extracting tarball to tmp_2211.16388...

 done.
Retrieving document from  https://arxiv.org/e-print/2211.16441


extracting tarball to tmp_2211.16441... done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.15698-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.15698) | **PHANGS-JWST First Results: Duration of the early phase of massive star  formation in NGC628**  |
|| Jaeyeon Kim, et al. -- incl., <mark>Frank Bigiel</mark>, <mark>Annie Hughes</mark>, <mark>Kathryn Kreckel</mark>, <mark>Janice C. Lee</mark>, <mark>Daizhong Liu</mark>, <mark>Eva Schinnerer</mark>, <mark>Rowan J. Smith</mark> |
|*Appeared on*| *2022-11-30*|
|*Comments*| *15 pages, 4 figures, 2 tables; resubmitted to ApJL after minor revisions*|
|**Abstract**| The earliest stages of star formation, when young stars are still deeply embedded in their natal clouds, represent a critical phase in the matter cycle between gas clouds and young stellar regions. Until now, the high-resolution infrared observations required for characterizing this heavily obscured phase (during which massive stars have formed, but optical emission is not detected) could only be obtained for a handful of the most nearby galaxies. One of the main hurdles has been the limited angular resolution of the Spitzer Space Telescope. With the revolutionary capabilities of the JWST, it is now possible to investigate the matter cycle during the earliest phases of star formation as a function of the galactic environment. In this Letter, we demonstrate this by measuring the duration of the embedded phase of star formation and the implied time over which molecular clouds remain inert in the galaxy NGC628 at a distance of 9.8Mpc, demonstrating that the cosmic volume where this measurement can be made has increased by a factor of $>100$ compared to Spitzer. We show that young massive stars remain embedded for $5.1_{-1.4}^{+2.7}$Myr ($2.3_{-1.4}^{+2.7}$Myr of which being heavily obscured), representing $\sim20\%$ of the total cloud lifetime. These values are in broad agreement with previous measurements in five nearby ($D < 3.5$Mpc) galaxies and constitute a proof of concept for the systematic characterization of the early phase of star formation across the nearby galaxy population with the PHANGS-JWST survey. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.15816-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.15816) | **The XQR-30 Metal Absorber Catalog: 778 Absorption Systems Spanning 2 < z  < 6.5**  |
|| <mark>Rebecca L. Davies</mark>, et al. -- incl., <mark>Sarah E. I. Bosman</mark>, <mark>Romain A. Meyer</mark> |
|*Appeared on*| *2022-11-30*|
|*Comments*| *29 pages and 14 figures including 2 appendices. Re-submitted to MNRAS after responding to referee's comments*|
|**Abstract**| Intervening metal absorption lines in the spectra of z > 6 quasars are fundamental probes of the ionization state and chemical composition of circumgalactic and intergalactic gas near the end of the reionization epoch. Large absorber samples are required to robustly measure typical absorber properties and to refine models of the synthesis, transport, and ionization of metals in the early Universe. The "Ultimate XSHOOTER legacy survey of quasars at z~5.8-6.6" (XQR-30) has obtained high signal-to-noise spectra of 30 luminous quasars, nearly quadrupling the existing sample of 12 high quality z~6 quasar spectra. We use this unprecedented sample to construct a catalog of 778 systems showing absorption in one or more of MgII (360 systems), FeII (184), CII (46), CIV (479), SiIV (127), and NV (13) which span 2 < z < 6.5. This catalog significantly expands on existing samples of z > 5 absorbers, especially for CIV and SiIV which are important probes of the ionizing photon background at high redshift. The sample is 50% (90%) complete for rest-frame equivalent widths W > 0.03AA (0.09AA). We publicly release the absorber catalog along with completeness statistics and a Python script to compute the absorption search path for different ions and redshift ranges. This dataset is a key legacy resource for studies of enriched gas from the era of galaxy assembly to cosmic noon, and paves the way for even higher redshift studies with the James Webb Space Telescope and 30m-class telescopes. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.16123-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.16123) | **Spectroscopic time series performance of the Mid-Infrared Instrument on  the JWST**  |
|| <mark>Jeroen Bouwman</mark>, et al. -- incl., <mark>Thomas P. Greene</mark>, <mark>Juergen Schreiber</mark>, <mark>Silvia Scheithauer</mark>, <mark>Michael Mueller</mark> |
|*Appeared on*| *2022-11-30*|
|*Comments*| *Submitted for review to PASP*|
|**Abstract**| We present here the first ever mid-infrared spectroscopic time series observation of the transiting exoplanet \object{L 168-9 b} with the Mid-Infrared Instrument (MIRI) on the James Webb Space Telescope. The data were obtained as part of the MIRI commissioning activities, to characterize the performance of the Low Resolution Spectroscopy (LRS) mode for these challenging observations. To assess the MIRI LRS performance, we performed two independent analyses of the data. We find that with a single transit observation we reached a spectro-photometric precision of $\sim$50 ppm in the 7-8~\micron~range at R=50, consistent with $\sim$25 ppm systematic noise. The derived band averaged transit depth is 524~$\pm$~15~ppm and 547~$\pm$~13~ppm for the two applied analysis methods, respectively, recovering the known transit depth to within 1~$\sigma$. The measured noise in the planet's transmission spectrum is approximately 15-20\% higher than random noise simulations over wavelengths $6.8 \lesssim \lambda \lesssim 11$ $\mu$m. We observed an excess noise at shorter wavelengths, for which possible causes are discussed. This performance was achieved with limited in-flight calibration data, demonstrating the future potential of MIRI for the characterization of exoplanet atmospheres. |

## Failed papers

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.16441-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.16441) | **Estimating effective wind speed from Gemini Planet Imager's adaptive  optics data using covariance maps**  |
|| Daniel M. Levinstein, et al. -- incl., <mark>Lisa Poyneer</mark> |
|*Appeared on*| *2022-11-30*|
|*Comments*| *7 pages, 6 figures*|
|**Abstract**| The Earth's turbulent atmosphere results in speckled and blurred images of astronomical objects when observed by ground based visible and near-infrared telescopes. Adaptive optics (AO) systems are employed to reduce these atmospheric effects by using wavefront sensors (WFS) and deformable mirrors. Some AO systems are not fast enough to correct for strong, fast, high turbulence wind layers leading to the wind butterfly effect, or wind-driven halo, reducing contrast capabilities in coronagraphic images. Estimating the effective wind speed of the atmosphere allows us to calculate the atmospheric coherence time. This is not only an important parameter to understand for site characterization but could be used to help remove the wind butterfly in post processing. Here we present a method for estimating the atmospheric effective wind speed from spatio-temporal covariance maps generated from pseudo open-loop (POL) WFS data. POL WFS data is used as it aims to reconstruct the full wavefront information when operating in closed-loop. The covariance maps show how different atmospheric turbulent layers traverse the telescope. Our method successfully recovered the effective wind speed from simulated WFS data generated with the soapy python library. The simulated atmospheric turbulence profiles consist of two turbulent layers of ranging strengths and velocities. The method has also been applied to Gemini Planet Imager (GPI) AO WFS data. This gives insight into how the effective wind speed can affect the wind-driven halo seen in the AO image point spread function. In this paper, we will present results from simulated and GPI WFS data. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.16388-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.16388) | **On the sociology and hierarchy of voids: a study of seven CAVITY nearby  galaxy voids and their dynamical CosmicFlows-3 environment**  |
|| H.M. Courtois, et al. -- incl., <mark>K. Kreckel</mark> |
|*Appeared on*| *2022-11-30*|
|*Comments*| *Submitted A\&A Nov 29, 2022 - AA/2022/45578*|
|**Abstract**| Context. The present study addresses a key question for our understanding of the relation between void galaxies and their environment: the relationship between luminous and dark matter in and around voids. Aims. To explore how empty of matter local Universe voids are, we study the full (dark+luminous) matter content of seven nearby cosmic voids that are fully contained within the CosmicFlows-3 volume. Methods. The cosmic voids matter density profiles are independently obtained using two different methods. They are built on one hand from the galaxy redshift space 2 points-correlation function and, on the other hand, using peculiar velocity gradients from the CosmicFlows-3 dataset. Results. The results are noticeable since when using the redshift survey, all voids show a radial positive gradient of galaxies, while based on the dynamical analysis, only three of these voids display a clear underdensity of matter in their center. Conclusions. It is the first time such a detailed observational analysis of voids is conducted, showing that void emptiness should be derived from dynamical information. Yet, from this limited study, the Hercules void is the best candidate for a local Universe pure "pristine volume" expanding in 3 directions with no dark matter located in that void. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.16239-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.16239) | **Simultaneous gas accretion onto a pair of giant planets: Impact on their  final mass and on the protoplanetary disk structure**  |
|| Camille Bergez-Casalou, <mark>Bertram Bitsch</mark>, Sean N. Raymond |
|*Appeared on*| *2022-11-30*|
|*Comments*| *Accepted to A&A*|
|**Abstract**| Several planetary systems are known to host multiple giant planets. However, when two giant planets are accreting from the same disk, it is unclear what effect the presence of the second planet has on the gas accretion process of both planets. In this paper we perform long-term 2D isothermal hydrodynamical simulations (over more than 0.5 Myrs) with the FARGO-2D1D code, considering two non-migrating planets accreting from the same gaseous disk. We find that the evolution of the planets' mass ratio depends on gap formation. However, in all cases, when the planets start accreting at the same time, they end up with very similar masses (0.9 $<m_{p,out}/m_{p,in}<$ 1.1 after 0.5 Myrs). Delaying the onset of accretion of one planet allows the planets' mass ratio to reach larger values initially, but they quickly converge to similar masses afterward (0.8 $<m_{p,out}/m_{p,in}<$ 2 in $10^5$ yrs). In order to reproduce the more diverse observed mass ratios of exoplanets, the planets must start accreting gas at different times, and their accretion must be stopped quickly after the beginning of runaway gas accretion (less than 0.5 Myrs), for example via disk dispersal. The evolution of the planets' mass ratio can have an important impact on the dynamics of the system and may constrain the formation history of Jupiter and Saturn. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error Could not find figure empty.eps</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.15665-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.15665) | **Planetary Exploration Horizon 2061 Report, Chapter 4: From planetary  exploration goals to technology requirements**  |
|| Jérémie Lasue, et al. -- incl., <mark>Heidi Hammel</mark>, <mark>Linda Spilker</mark> |
|*Appeared on*| *2022-11-30*|
|*Comments*| *83 pages, 23 figures, 7 tables, Horizon 2061 is a science-driven, foresight exercise, for future scientific investigations*|
|**Abstract**| This chapter reviews for each province and destination of the Solar System the representative space missions that will have to be designed and implemented by 2061 to address the six key science questions about the diversity, origins, workings and habitability of planetary systems (described in chapter 1) and to perform the critical observations that have been described in chapters 3 and partly 2. It derives from this set of future representative missions, some of which will have to be flown during the 2041-2061 period, the critical technologies and supporting infrastructures that will be needed to fly these challenging missions, thus laying the foundation for the description of technologies and infrastructures for the future of planetary exploration that is given in chapters 5 and 6, respectively. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.15781-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.15781) | **3UCubed: The IMAP Student Collaboration CubeSat Project**  |
|| Marcus Alfred, et al. -- incl., <mark>Sonya Smith</mark>, <mark>Walter Foster</mark>, <mark>Lindsay Bartolone</mark> |
|*Appeared on*| *2022-11-30*|
|*Comments*| **|
|**Abstract**| The 3UCubed project is a 3U CubeSat being jointly developed by the University of New Hampshire, Sonoma State University, and Howard University as a part of the NASA Interstellar Mapping and Acceleration Probe, IMAP, student collaboration. This project comprises of a multidisciplinary team of undergraduate students from all three universities. The mission goal of the 3UCubed is to understand how Earths polar upper atmosphere the thermosphere in Earths auroral regions, responds to particle precipitation and solar wind forcing, and internal magnetospheric processes. 3UCubed includes two instruments with rocket heritage to achieve the science mission: an ultraviolet photomultiplier tube, UVPMT, and an electron retarding potential analyzer ERPA. The spacecraft bus consists of the following subsystems: Attitude Determination and Control, Command and Data Handling, Power, Communication, Structural, and Thermal. Currently, the project is in the post-PDR stage, starting to build and test engineering models to develop a FlatSat prior to critical design review in 2023. The goal is to launch at least one 3U CubeSat to collect science data close to the anticipated peak of Solar Cycle 25 around July 2025. Our mother mission, IMAP, is also projected to launch in 2025, which will let us jointly analyze the science data of the main mission, providing the solar wind measurements and inputs to the magnetosphere with that of 3UCubed, providing the response of Earths cusp to these inputs. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2211.15698.md
    + _build/html/tmp_2211.15698/./fig2.png
    + _build/html/tmp_2211.15698/./app_fig1.png
    + _build/html/tmp_2211.15698/./fig1.png
exported in  _build/html/2211.15816.md
    + _build/html/tmp_2211.15816/./spike_filtering.png
    + _build/html/tmp_2211.15816/./velocity_mismatch_filtering.png
    + _build/html/tmp_2211.15816/./ew_filtering.png
    + _build/html/tmp_2211.15816/./SDSSJ2310+1855_5.93_offset_vs_wavelength.png
    + _build/html/tmp_2211.15816/./ULASJ0148+0600_z2.7220.png
exported in  _build/html/2211.16123.md
    + _build/html/tmp_2211.16123/./figures/miritso_rateints_b.png
    + _build/html/tmp_2211.16123/./figures/TSO_white_light_curve_bg.png
    + _build/html/tmp_2211.16123/figures/L168-9b_transit_systematics_and_residual_MIRI_LRS.png
    + _build/html/tmp_2211.16123/figures/L168-9b_uncertainty_estimate.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\vdag$}{(v)^\dagger}$
$\newcommand$
$\newcommand$</div>



<div id="title">

# PHANGS-JWST First Results: Duration of the early phase of massive star formation in NGC 628

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2211.15698-b31b1b.svg)](https://arxiv.org/abs/2211.15698)<mark>Appeared on: 2022-11-30</mark> - _15 pages, 4 figures, 2 tables; resubmitted to ApJL after minor revisions_

</div>
<div id="authors">

Jaeyeon Kim, et al. -- incl., <mark>Frank Bigiel</mark>, <mark>Annie~Hughes</mark>, <mark>Kathryn Kreckel</mark>, <mark>Janice C. Lee</mark>, <mark>Daizhong Liu</mark>, <mark>Eva Schinnerer</mark>, <mark>Rowan~J.~Smith</mark>

</div>
<div id="abstract">

**Abstract:** The earliest stages of star formation, when young stars are still deeply embedded in their natal clouds, represent a critical phase in the matter cycle between gas clouds and young stellar regions. Until now, the high-resolution infrared observations required for characterizing this heavily obscured phase (during which massive stars have formed, but optical emission is not detected) could only be obtained for a handful of the most nearby galaxies. One of the main hurdles has been the limited angular resolution of the\textit{Spitzer Space Telescope}. With the revolutionary capabilities of the JWST, it is now possible to investigate the matter cycle during the earliest phases of star formation as a function of the galactic environment. In this Letter, we demonstrate this by measuring the duration of the embedded phase of star formation and the implied time over which molecular clouds remain inert in the galaxy NGC 628 at a distance of 9.8 Mpc, demonstrating that the cosmic volume where this measurement can be made has increased by a factor of$>100$compared to\textit{Spitzer}. We show that young massive stars remain embedded for$5.1_{-1.4}^{+2.7}$Myr ($2.3_{-1.4}^{+2.7}$Myr of which being heavily obscured), representing$\sim 20 \% $of the total cloud lifetime. These values are in broad agreement with previous measurements in five nearby ($D < 3.5$Mpc) galaxies and constitute a proof of concept for the systematic characterization of the early phase of star formation across the nearby galaxy population with the PHANGS--JWST survey.

</div>

<div id="div_fig1">

<img src="tmp_2211.15698/./fig2.png" alt="Fig4" width="100%"/>

**Figure 4. -** The left panel shows the measured deviation of gas-to-SFR tracer (CO-to-21 $\mu$m) flux ratios compared to the galactic average as a function of the size of apertures centered on CO and 21 $\mu$m emission peaks. The data underlying this figure can be found in Appendix \ref{app:data}. The error bars show the 1$\sigma$ uncertainty of each measurement whereas the shaded region within the error bar indicates the effective 1$\sigma$ error, considering the covariance between data points. Our best-fitting model (dashed line), as well as the galactic average (horizontal line) are also shown. The constrained $\lambda$ is indicated with a downward arrow and other best-fitting parameters ($t_{\rm 21 \mu m}$ and $t_{\rm fb, 21 \mu m}$) are listed in Table \ref{tab:result}. The right panel illustrates the evolutionary sequence from inert molecular clouds to embedded star formation, partially exposed star formation, and finally to fully revealed young stellar regions. The duration of the CO emitting phase ($t_{\rm CO}$) is shown in orange while the time during which 21 $\mu$m and H$\alpha$ emission are detected without associated CO emission are shown in dark blue and dark purple, respectively. The feedback timescale, which is the time for which both CO and a SFR tracer are found coincident, is shown in light blue (for 21 $\mu$m) and light purple (for H$\alpha$). The error bars indicate the 1$\sigma$ uncertainty for each measurement. (*fig:result*)

</div>
<div id="div_fig2">

<img src="tmp_2211.15698/./app_fig1.png" alt="Fig2" width="100%"/>

**Figure 2. -** The top panel shows the flux contrast ($\rm{\delta}log_{10}\mathcal{F}$) used to identify peaks on CO (blue) 21 $\mu$m (red) map as a function of the average filling factor $\zeta$. The bottom panel shows the ratio of the feedback timescale ($t_{\rm fb, 21\mu m}$) and the total duration of the GMC lifecycle ($\tau$) as a function of $\zeta$. In both panels, the shaded region shows the parameter space where crowding of sources can lead to an overestimation of the feedback timescale. Our data points are well outside of this region, confirming that we sufficiently resolve star-forming regions and our measurement of $t_{\rm fb, 21 \mu m}$ is reliable.    (*fig:blending*)

</div>
<div id="div_fig3">

<img src="tmp_2211.15698/./fig1.png" alt="Fig3" width="100%"/>

**Figure 3. -** \textit{Top:} Comparison between the \textit{Spitzer} 24 $\mu$m map (left) and the JWST 21 $\mu$m map (middle), which has 10 times better resolution (0.67$\arcsec$) compared to \textit{Spitzer}(6$\arcsec$). Orange circles show the beam in each panel. \textit{Bottom:} Composite three color images obtained by combining CO (blue), 21 $\mu$m (green), and H$\alpha$(red). The bottom right panel shows the zoomed-in image of the white rectangular region marked in the bottom left panel, with symbols indicating the distribution of emission peaks using the same color scheme. Emission peaks of CO (indicated with $\textcolor{blue}{\times}$), 21 $\mu$m ($\textcolor{green}{+}$), and H$\alpha$($\textcolor{red}{\triangle}$) show spatial offsets, indicating that these represent distinctive stages of star formation. The JWST field of view is outlined in green. The crowded galactic center (white circle), as well as extremely bright star-forming regions (yellow circles) are excluded from our analysis (see text). (*fig:map*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\NV}{\mbox{N \textsc{v}}}$
$\newcommand{\SiIV}{\mbox{Si \textsc{iv}}}$
$\newcommand{\CIV}{\mbox{C \textsc{iv}}}$
$\newcommand{\CII}{\mbox{C \textsc{ii}}}$
$\newcommand{\MgII}{\mbox{Mg \textsc{ii}}}$
$\newcommand{\SII}{\mbox{S \textsc{ii}}}$
$\newcommand{\ZnII}{\mbox{Zn \textsc{ii}}}$
$\newcommand{\TiII}{\mbox{Ti \textsc{ii}}}$
$\newcommand{\FeII}{\mbox{Fe \textsc{ii}}}$
$\newcommand{\AlII}{\mbox{Al \textsc{ii}}}$
$\newcommand{\AlIII}{\mbox{Al \textsc{iii}}}$
$\newcommand{\SiII}{\mbox{Si \textsc{ii}}}$
$\newcommand{\OI}{\mbox{O \textsc{i}}}$
$\newcommand{\HI}{\mbox{H \textsc{i}}}$
$\newcommand{\CoII}{\mbox{Co \textsc{ii}}}$
$\newcommand{\CrII}{\mbox{Cr \textsc{ii}}}$
$\newcommand{\CaII}{\mbox{Ca \textsc{ii}}}$
$\newcommand{\MnII}{\mbox{Mn \textsc{ii}}}$
$\newcommand{\NaI}{\mbox{Na \textsc{i}}}$
$\newcommand{\MgI}{\mbox{Mg \textsc{i}}}$
$\newcommand{\Lya}{Ly\alpha}$
$\newcommand{\numCIV}{479}$
$\newcommand{\numMgII}{360}$
$\newcommand{\numCII}{46}$
$\newcommand{\kms}{ km s^{-1}}$
$\newcommand{\bibtex}{\textsc{Bib}\!\TeX}$
$\newcommand{\appropto}{\mathrel{\vcenter{$
$  \offinterlineskip\halign{\hfil##\cr$
$    \propto\cr\noalign{\kern2pt}\sim\cr\noalign{\kern-2pt}}}}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\NV$}{\mbox{N $\textsc${v}}}$
$\newcommand{$\SiIV$}{\mbox{Si $\textsc${iv}}}$
$\newcommand{$\CIV$}{\mbox{C $\textsc${iv}}}$
$\newcommand{$\CII$}{\mbox{C $\textsc${ii}}}$
$\newcommand{$\MgII$}{\mbox{Mg $\textsc${ii}}}$
$\newcommand{$\SII$}{\mbox{S $\textsc${ii}}}$
$\newcommand{$\ZnII$}{\mbox{Zn $\textsc${ii}}}$
$\newcommand{$\TiII$}{\mbox{Ti $\textsc${ii}}}$
$\newcommand{$\FeII$}{\mbox{Fe $\textsc${ii}}}$
$\newcommand{$\AlII$}{\mbox{Al $\textsc${ii}}}$
$\newcommand{$\AlII$I}{\mbox{Al $\textsc${iii}}}$
$\newcommand{$\SiII$}{\mbox{Si $\textsc${ii}}}$
$\newcommand{$\OI$}{\mbox{O $\textsc${i}}}$
$\newcommand{$\HI$}{\mbox{H $\textsc${i}}}$
$\newcommand{$\CoII$}{\mbox{Co $\textsc${ii}}}$
$\newcommand{$\CrII$}{\mbox{Cr $\textsc${ii}}}$
$\newcommand{$\CaII$}{\mbox{Ca $\textsc${ii}}}$
$\newcommand{$\MnII$}{\mbox{Mn $\textsc${ii}}}$
$\newcommand{$\NaI$}{\mbox{Na $\textsc${i}}}$
$\newcommand{$\MgI$}{\mbox{Mg $\textsc${i}}}$
$\newcommand{$\Lya$}{Ly\alpha}$
$\newcommand{$\numCIV$}{479}$
$\newcommand{$\numMgII$}{360}$
$\newcommand{$\numCII$}{46}$
$\newcommand{$\kms$}{ km s^{-1}}$
$\newcommand{$\bibtex$}{$\textsc${Bib}\!\TeX}$
$\newcommand{\appropto}{\mathrel{\vcenter{$
$  \offinterlineskip\halign{\hfil##\cr$
$    \propto\cr\noalign{\kern2pt}\sim\cr\noalign{\kern-2pt}}}}}$</div>



<div id="title">

#  6.5

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2211.15816-b31b1b.svg)](https://arxiv.org/abs/2211.15816)<mark>Appeared on: 2022-11-30</mark> - _29 pages and 14 figures including 2 appendices. Re-submitted to MNRAS after responding to referee's comments_

</div>
<div id="authors">

<mark><mark>Rebecca L. Davies</mark></mark>, et al. -- incl., <mark><mark>Sarah E. I. Bosman</mark></mark>, <mark><mark>Romain A. Meyer</mark></mark>

</div>
<div id="abstract">

**Abstract:** Intervening metal absorption lines in the spectra of$z\gtrsim$6 quasars are fundamental probes of the ionization state and chemical composition of circumgalactic and intergalactic gas near the end of the reionization epoch. Large absorber samples are required to robustly measure typical absorber properties and to refine models of the synthesis, transport, and ionization of metals in the early Universe. The "Ultimate XSHOOTER legacy survey of quasars at$z\sim$5.8-6.6"\mbox{(XQR-30)}has obtained high signal-to-noise spectra of 30 luminous quasars, nearly quadrupling the existing sample of 12 high quality$z\sim$6 quasar spectra. We use this unprecedented sample to construct a catalog of 778 systems showing absorption in one or more of$\MgII$($\numMgII$systems),$\FeII$(184),$\CII$($\numCII$),$\CIV$($\numCIV$),$\SiIV$(127), and$\NV$(13) which span\mbox{2 $\lesssim z \lesssim$ 6.5}. This catalog significantly expands on existing samples of\mbox{$z \gtrsim$ 5}absorbers, especially for$\CIV$and$\SiIV$which are important probes of the ionizing photon background at high redshift. The sample is 50\%(90\%) complete for rest-frame equivalent widths$W \gtrsim$0.03\AA(0.09\AA). We publicly release the absorber catalog along with completeness statistics and a$\textsc${Python}script to compute the absorption search path for different ions and redshift ranges. This dataset is a key legacy resource for studies of enriched gas from the era of galaxy assembly to cosmic noon, and paves the way for even higher redshift studies with the\textit{James Webb Space Telescope}and 30m-class telescopes.

</div>

<div id="div_fig1">

<img src="tmp_2211.15816/./spike_filtering.png" alt="Fig1.1" width="33%"/><img src="tmp_2211.15816/./velocity_mismatch_filtering.png" alt="Fig1.2" width="33%"/><img src="tmp_2211.15816/./ew_filtering.png" alt="Fig1.3" width="33%"/>

**Figure 1. -** Illustration of the criteria used to filter the lists of candidate absorption pairs. The absorption profile of each transition is resampled onto a 10 $\kms$ grid centered at the redshift of the candidate. \textit{Top}: Line 2 is a skyline residual. A candidate is rejected if either absorption line does not span at least three 10 $\kms$ velocity channels (about one resolution element; Section \ref{subsubsec:skylines}). \textit{Middle and Bottom}: In each candidate, both transitions are real absorption lines but they are associated with unrelated absorption systems. In the middle panel, the absorption profiles are misaligned in velocity space. We reject candidates that do not have significant absorption overlapping between the two transitions (Section \ref{subsubsec:overlap_signal}). The candidate in the bottom panel is rejected because the observed equivalent widths (black) are inconsistent with Voigt profile fits assuming a single column density (red) (Section \ref{subsubsec:ew_test}).  (*fig:filtering*)

</div>
<div id="div_fig2">

<img src="tmp_2211.15816/./SDSSJ2310+1855_5.93_offset_vs_wavelength.png" alt="Fig8" width="100%"/>

**Figure 8. -** Left: Difference between the centroid velocities of VIS and NIR transitions (blue and red markers, respectively) as a function of observed wavelength for a single-component low-ionization absorption system at $z$ = 5.939 in the spectrum of SDSSJ2310+1855. The velocities are relative to the redshift of the absorption system in the final catalog (grey dashed line). The NIR transitions are offset by a median velocity of 17 $\kms$(red dashed line). Right: Velocity offsets after applying the 17 $\kms$ shift to the NIR spectrum. There is no evidence for any systematic variation in the measured centroid velocity as a function of wavelength. The final centroid velocities vary by $\sim$ 10 $\kms$ which is the width of a spectral pixel. (*fig:dv_wavelength*)

</div>
<div id="div_fig3">

<img src="tmp_2211.15816/./ULASJ0148+0600_z2.7220.png" alt="Fig2" width="100%"/>

**Figure 2. -** Four component fit to a low-ionization absorption system at \mbox{$z$ = 2.72}. Each color indicates a different kinematic component. The $\MgII$ profiles (rows 2 and 3) show a blueshifted wing that is not detected in the other ions. The $\FeII$ profiles (rows 4 and 5) are well fit by two components (orange and red). However, the corresponding central absorption trough in $\MgII$(rows 2 and 3) can only be fit with a single component (brown) due to saturation. The $\MgI$ absorption profile (row 1) is fit using the same central component (brown) because the low S/N prevents a robust multi-component decomposition. (*fig:multi_comp_fitting*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\vdag$}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$</div>



<div id="title">

# Spectroscopic time series performance of the Mid-Infrared Instrument on the JWST

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2211.16123-b31b1b.svg)](https://arxiv.org/abs/2211.16123)<mark>Appeared on: 2022-11-30</mark> - _Submitted for review to PASP_

</div>
<div id="authors">

<mark>Jeroen Bouwman</mark>, et al. -- incl., <mark>Thomas P. Greene</mark>, <mark>J\"{u}rgen Schreiber</mark>, <mark>Silvia Scheithauer</mark>, <mark>Michael Mueller</mark>

</div>
<div id="abstract">

**Abstract:** We present here the first ever mid-infrared spectroscopic time series observation of the transiting exoplanet$\object${L 168-9 b}with the Mid-Infrared Instrument (MIRI) on the James Webb Space Telescope. The data were obtained as part of the MIRI commissioning activities, to characterize the performance of the Low Resolution Spectroscopy (LRS) mode for these challenging observations. To assess the MIRI LRS performance, we performed two independent analyses of the data. We find that with a single transit observation we reached a spectro-photometric precision of$\sim$50 ppm in the 7-8\micronrange at R=50, consistent with$\sim$25 ppm systematic noise. The derived band averaged transit depth is 524$\pm$15 ppm and 547$\pm$13 ppm for the two applied analysis methods, respectively, recovering the known transit depth to within 1$\sigma$. The measured noise in the planet's transmission spectrum is approximately 15-20\%higher than random noise simulations over wavelengths$6.8 \lesssim \lambda \lesssim 11$$\mu$m. We observed an excess noise at shorter wavelengths, for which possible causes are discussed. This performance was achieved with limited in-flight calibration data, demonstrating the future potential of MIRI for the characterization of exoplanet atmospheres.

</div>

<div id="div_fig1">

<img src="tmp_2211.16123/./figures/miritso_rateints_b.png" alt="Fig2.1" width="50%"/><img src="tmp_2211.16123/./figures/TSO_white_light_curve_bg.png" alt="Fig2.2" width="50%"/>

**Figure 2. -** Level 2a calibrated spectral images and band averaged light curve of the $\object${L 168-9} system. The images are rotated by 90 degrees for display purposes and the spectra are shown in the full SLITLESSPRISM subarray, which measures 416 rows $\times$ 72 columns on the detector. In this orientation, the wavelength increases from right to left. The images were generated from the seg-002 \texttt{rateints} file, which is one of 5 segments that make up the exposure. Both images show the same spectral image: (top) with a linear scaling; and (bottom) with a log scaling and limits adjusted to bring out the complex diffraction patterns and scattered light seen at the short wavelengths. The normalized lightcurve is obtained by summing the signal seen in the spectral images between detector rows 230 to 380 and detector columns 20 to 51 and dividing the total signal per integration by the averages signal in time. Even in this very basic calibrated data the planetary transit can be clearly observed. (*fig:rateints_ims*)

</div>
<div id="div_fig2">

<img src="tmp_2211.16123/figures/L168-9b_transit_systematics_and_residual_MIRI_LRS.png" alt="Fig4" width="100%"/>

**Figure 4. -** \texttt{CASCADe}'s white-light (4.85--11.85 $\mu$m) light curve analysis.  The top panel shows the uncalibrated white light curve (blue data points) together with the fitted band averaged systematics model (black data points). The middle panel show the calibrated data together with the fitted light curve model. The lower panel shows the residuals from subtracting the lightcurve model from the calibrated time series data.  In the analysis the first 30 minutes of the observations have been omitted as they sow the strongest response drifts (which can be seen in Figure \ref{fig:rateints_ims}).
\label{fig:fig_LC} (*fig:fig_LC*)

</div>
<div id="div_fig3">

<img src="tmp_2211.16123/figures/L168-9b_uncertainty_estimate.png" alt="Fig9" width="100%"/>

**Figure 9. -** Noise estimates of the transit spectrum of L 168-9 b. The blue dots show \texttt{CASCADe}'s noise estimate while the green routs show \texttt{Eureka!}'s 1 $\sigma$ uncertainty estimate. The red curve shows the expected noise limit for this observations using a scaled photon conversion efficiency \citep[][see paragraph \ref{sec:analysis}]{2015ApJ...805...23C}. The black curve shows the estimated noise limit using MIRISim \citep{mirisim} and the in orbit determined \texttt{PHOTOM} and wavelength calibrating files (see also section \ref{sec:analysis}).
\label{fig:fig_error_analysis} (*fig:fig_error_analysis*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

112  publications in the last 7 days.
	 _build/html/2211.16123.md
	 _build/html/2211.15816.md
	 _build/html/2211.15698.md
	 _build/html/2211.15576.md
	 _build/html/2211.15434.md
	 _build/html/2211.15288.md
	 _build/html/2211.15224.md
	 _build/html/2211.14543.md
	 _build/html/2211.14330.md
	 _build/html/2211.14322.md
	 _build/html/2211.14132.md
	 _build/html/2211.13811.md
	 _build/html/2211.13426.md
	 _build/html/2211.13318.md
	 _build/html/2211.13146.md
	 _build/html/2211.12613.md
	 _build/html/2211.11767.md
	 _build/html/2211.09840.md
	 _build/html/2211.09217.md
	 _build/html/2211.08493.md
	 _build/html/2211.08487.md
	 _build/html/2211.08294.md
	 _build/html/2211.07667.md
	 _build/html/2211.07659.md
	 _build/html/2211.06454.md
	 _build/html/2211.06205.md
	 _build/html/2211.06194.md
	 _build/html/2211.05022.md
	 _build/html/2211.04048.md
	 _build/html/2211.03641.md
	 _build/html/2211.02502.md
	 _build/html/2211.01474.md
	 _build/html/2211.00667.md
	 _build/html/2211.00657.md
	 _build/h

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers